In [1]:
import pandas as pd
df=pd.read_csv("abalone.csv",header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [2]:
df=df.rename(columns={0:"sex",1:"length",2:"diameter",3:"height",4:"whole_weight",5:"shucked_weight",6:"viscera_weight",7:"shell_weight",8:"rings"})
df.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [3]:
y=df[["rings"]]
y.groupby("rings")["rings"].count()

rings
1       1
2       1
3      15
4      57
5     115
6     259
7     391
8     568
9     689
10    634
11    487
12    267
13    203
14    126
15    103
16     67
17     58
18     42
19     32
20     26
21     14
22      6
23      9
24      2
25      1
26      1
27      2
29      1
Name: rings, dtype: int64

In [4]:
y=y.replace([1,2,3,4,5,6,7,8],1)

In [5]:
y=y.replace([9,10],2)

In [6]:
y=y.replace([11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,29],3)

In [10]:
import numpy as np
np.unique(y)

array([1, 2, 3])

In [11]:
y.groupby("rings")["rings"].count()

rings
1    1407
2    1323
3    1447
Name: rings, dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder
sex=df[["sex"]]
le=LabelEncoder()
sex=pd.DataFrame(le.fit_transform(sex),columns=sex.columns)
sex.head()

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,sex
0,2
1,2
2,0
3,2
4,1


In [13]:
sub_X=df.drop(["sex","rings"],axis=1)
X=pd.concat([sex,sub_X],axis=1)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight
0,1.151980,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217
1,1.151980,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987
2,-1.280690,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139
3,1.151980,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294
4,-0.064355,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757


In [195]:
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
7,1455.940503
2,1397.190901
1,1304.782954
4,1167.074609
6,1086.363631
3,1062.219850
5,793.567670
0,2.447641


In [206]:
X_scores=scores[scores["scores"]>100]
len(X_scores)

7

In [207]:
X_prime=X.iloc[:,X_scores.index]
X_prime.head()

,shell_weight,diameter,length,whole_weight,viscera_weight,height,shucked_weight
0,-0.638217,-0.432149,-0.574558,-0.641898,-0.726212,-1.064424,-0.607685
1,-1.212987,-1.439929,-1.448986,-1.230277,-1.205221,-1.183978,-1.170910
2,-0.207139,0.122130,0.050033,-0.309469,-0.356690,-0.107991,-0.463500
3,-0.602294,-0.432149,-0.699476,-0.637819,-0.607600,-0.347099,-0.648238
4,-1.320757,-1.540707,-1.615544,-1.272086,-1.287337,-1.423087,-1.215968


In [208]:
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)

(3132, 7)
(1045, 7)


In [209]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(n_estimators=250,criterion="entropy",n_jobs=-1,class_weight="balanced",ccp_alpha=0.0035,
                              random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(ccp_alpha=0.0035, class_weight='balanced',
                       criterion='entropy', n_estimators=250, n_jobs=-1,
                       random_state=0)

In [210]:
from sklearn.metrics import accuracy_score
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.7254150702426565
Accuracy on test data:  0.661244019138756


In [225]:
Gradient=GradientBoostingClassifier(n_estimators=200,learning_rate=0.02,criterion="mse",max_features="sqrt",
                                    random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Gradient

GradientBoostingClassifier(criterion='mse', learning_rate=0.02,
                           max_features='sqrt', n_estimators=200,
                           random_state=0)

In [226]:
train_pred=Gradient.predict(X_train)
test_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6973180076628352
Accuracy on test data:  0.6373205741626794


In [247]:
from sklearn import svm
SVM=svm.SVC(C=3,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
SVM

SVC(C=3, random_state=0)

In [248]:
train_pred=SVM.predict(X_train)
test_pred=SVM.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6711366538952745
Accuracy on test data:  0.6507177033492823


In [256]:
sub_X=df.drop(["sex","rings"],axis=1)
X=pd.concat([sex,sub_X],axis=1)

In [257]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight
0,1.0,0.513514,0.521008,0.084071,0.181335,0.150303,0.132324,0.147982
1,1.0,0.371622,0.352941,0.079646,0.079157,0.066241,0.063199,0.068261
2,0.0,0.614865,0.613445,0.119469,0.239065,0.171822,0.185648,0.207773
3,1.0,0.493243,0.521008,0.110619,0.182044,0.144250,0.149440,0.152965
4,0.5,0.344595,0.336134,0.070796,0.071897,0.059516,0.051350,0.053313


In [258]:
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
7,1455.940503
2,1397.190901
1,1304.782954
4,1167.074609
6,1086.363631
3,1062.219850
5,793.567670
0,2.447641


In [259]:
X_scores=scores[scores["scores"]>100]
len(X_scores)

7

In [260]:
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)

(3132, 7)
(1045, 7)


In [261]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(n_estimators=250,criterion="entropy",n_jobs=-1,class_weight="balanced",ccp_alpha=0.0035,
                              random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Forest

RandomForestClassifier(ccp_alpha=0.0035, class_weight='balanced',
                       criterion='entropy', n_estimators=250, n_jobs=-1,
                       random_state=0)

In [262]:
train_pred=Forest.predict(X_train)
test_pred=Forest.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.7254150702426565
Accuracy on test data:  0.661244019138756


In [263]:
Gradient=GradientBoostingClassifier(n_estimators=200,learning_rate=0.02,criterion="mse",max_features="sqrt",
                                    random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Gradient

GradientBoostingClassifier(criterion='mse', learning_rate=0.02,
                           max_features='sqrt', n_estimators=200,
                           random_state=0)

In [264]:
train_pred=Gradient.predict(X_train)
test_pred=Gradient.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6973180076628352
Accuracy on test data:  0.6373205741626794


In [265]:
SVM=svm.SVC(C=3,random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
SVM

SVC(C=3, random_state=0)

In [266]:
train_pred=SVM.predict(X_train)
test_pred=SVM.predict(X_test)
print("Accuracy on train data: ",accuracy_score(y_train,train_pred))
print("Accuracy on test data: ",accuracy_score(y_test,test_pred))

Accuracy on train data:  0.6711366538952745
Accuracy on test data:  0.6507177033492823


### There is no difference between the results with StandardScaler and results with MinMaxScaler.